In [1]:
import numpy as np;
import matplotlib.pyplot as plt;

### Oppgave 1)
## a)

## b)

In [2]:
def createPolymer(N):
    polymer = np.zeros((N, 2)) 
    polymer[:, 1] = N // 2                                     # setter y-koordinat
    polymer[:, 0] = np.array([i for i in range(N)])          # setter x-koordinat
    return polymer

## c)

## d)

In [3]:
def illustrationPolymer(polymer):
    N = len(polymer)                 
    grid = np.zeros((N+1,N+1))        # Lager (N+1)*(N+1) grid
    grid -= int(N/2)                         # Setter bakgrunnsverdien til å være -N for å få synlighet blant lave N
    for monomerNumber in range(N):
        x = int(polymer[monomerNumber, 0])  
        y = int(polymer[monomerNumber, 1])
        grid[y,x] = monomerNumber + 1
    plt.pcolormesh(grid)
    plt.show()

## e)